In [ ]:
%load_ext autoreload
%autoreload 2
import sys
# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [ ]:
from src import landtype_buffer
from src import landtype_buffer_trial
from src import gpd_extract_lons_lats
import pandas as pd
import numpy as np
import multiprocessing
import geopandas as gpd
import pygeohydro as gh

In [ ]:
import os,rasterio,pyproj

In [ ]:
def geometry(nlcd_df_dummy):
    geo_gdf = gpd.GeoDataFrame(
    nlcd_df_dummy,
    geometry=gpd.points_from_xy(nlcd_df_dummy['lon'], nlcd_df_dummy['lat']),
    crs=4326)   
    return geo_gdf


def read_data():
    nlcd_df = pd.read_csv('/home/aevans/Kara_HW/data/nlcd_df.csv')
    nlcd_df_dummy = nlcd_df.iloc[0:25]
    geo_gdf = geometry(nlcd_df_dummy)
    return geo_gdf


def parallelize_get_landtype(df_split, buffer_size=90):
    mode_list = []
    # print("df_split",df_split)
    
    for i,site in df_split.iterrows():
        sorted_df = df_split.sort_index(ascending=True)
        buffer_df = landtype_buffer.landtype_buffer(df_split, buffer_size)
        print(site["station"])
        buffer_df.to_parquet(f'/home/aevans/Kara_HW/data/buffer_{site["station"]}.parquet')


def main():
    geo_gdf = read_data()
    # print(geo_gdf)
    num_cores = 5  #leave one free to not freeze machine
    df_split = np.array_split(geo_gdf, 5)
    # print(df_split)
    pool = multiprocessing.Pool(num_cores)
    pool.map(parallelize_get_landtype, df_split)
    pool.close()

In [ ]:
main()